## <font color=red>较为复杂，可以先跳过</font>

下面的是已经实现了相关功能的  
可以直接通过改变data获取到不同歌曲的数据

## 网易云评论数据抓取

- 找到未加密的参数
- 想办法把参数进行加密（必须参考网易的逻辑），params,encSecKsy
- 请求到网易，拿到评论

- 使用前端断点调试，找到对应的加密代码  
![](img/2022-09-29-00-37-37.png)
![](img/2022-09-29-00-34-00.png)

发现是<font color=yellow>window.arsea(参数, xxxx,xxxx,xxxx)</font>进行的加密  
我们需要的param => encText, encSecKey => encSecKey

In [32]:
import requests

# 拿取评论的url
url = "https://music.163.com/weapi/comment/resource/comments/get?csrf_token="

# 真实参数
data = {
    "csrf_token": "",
    "cursor": "-1",
    "offset": "0",
    "orderType": "1",
    "pageNo": "1",
    "pageSize": "20",
    "rid": "R_SO_4_1330348068",
    "threadId": "R_SO_4_1330348068"
}

# 处理加密过程
# window.arsea加密过程为如下,使用d函数进行加密
# 搜索找到window.arsea对应的加密函数
'''
    function a(a) {
        var d, e, b = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789", c = "";
        for (d = 0; a > d; d += 1)
            e = Math.random() * b.length,
            e = Math.floor(e),
            c += b.charAt(e);
        return c
    }
    function b(a, b) {
        var c = CryptoJS.enc.Utf8.parse(b)
          , d = CryptoJS.enc.Utf8.parse("0102030405060708")
          , e = CryptoJS.enc.Utf8.parse(a)
          , f = CryptoJS.AES.encrypt(e, c, {
            iv: d,
            mode: CryptoJS.mode.CBC
        });
        return f.toString()
    }
    function c(a, b, c) {
        var d, e;
        return setMaxDigits(131),
        d = new RSAKeyPair(b,"",c),
        e = encryptedString(d, a)
    }
    function d(d, e, f, g) {
        var h = {}
          , i = a(16);
        return h.encText = b(d, g),
        h.encText = b(h.encText, i),
        h.encSecKey = c(i, e, f),
        h
    }
    window.asrsea = d,
'''

'''
    window.asrsea(JSON.stringify(i7b), buU5Z(["流泪", "强"]), buU5Z(Rg1x.md), buU5Z(["爱心", "女孩", "惊恐", "大笑"]));
    对于其中一些，可以直接在控制台中运行获取到，比如：buU5Z(["流泪", "强"])
    注意在控制中获取时要设计合适的断点
    解析各个参数
    d: 数据，
    e: '010001',
    f: '00e0b509f6259df8642dbc35662901477df22677ec152b5ff68ace615bb7\
        b725152b3ab17a876aea8a5aa76d2e417629ec4ee341f56135fccf695280\
        104e0312ecbda92557c93870114af6c9d05c4f7f0c3685b7a46bee2559325\
        75cce10b424d813cfe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7',
    g: '0CoJUm6Qyw8W8jud'
'''

# 定义e,f,g
e = '010001'
f = '00e0b509f6259df8642dbc35662901477df22677ec152b5ff68ace615bb7\
b725152b3ab17a876aea8a5aa76d2e417629ec4ee341f56135fccf695280\
104e0312ecbda92557c93870114af6c9d05c4f7f0c3685b7a46bee2559325\
75cce10b424d813cfe4875d3e82047b97ddef52741d546b8e289dc6935b3ece0462db0a22b8e7'
g = '0CoJUm6Qyw8W8jud'

# 分析上诉代码不难得知，当i不变时，得到的encSecKey也是一个定值
# 所以下面直接通过固定i值，得到对应的encSecKey，从而解决一个参数
i = "WUIAKrEtcMVOmiqE"
encSecKey = "2b06b576cece434ae4bf65341bd3717d3043349099a854246e14492a4508947be3\
e153ba0ff0e061397d104371d34f23e6b0c7f36420acbf8f277ee6a531ae6ba7db3127d4419b92d\
88229f4d6a98de5bcd97a67d229201a9f0b4f168add0d48b17eafb225bacf17f2066128f9d78007e087af8202d538170b4f0634837aed42"

# 同理，经过对i的确定，encText中很多参数也固定了
# 变化的是传入时的data
# 用于得到对应的params
from Crypto.Cipher import AES
from base64 import b64encode


def get_params(data):# 默认这里接受的是字符串
    first = enc_params(data, g)
    second = enc_params(first, i)
    return second


# 用于解决AES中16倍数加密逻辑的问题
def to_16(data):
    pad = 16 - len(data) % 16
    data += chr(pad) * pad
    return data


def enc_params(data, key):
    iv = "0102030405060708"
    data = to_16(data)
    # 创建加密器
    aes = AES.new(key=key.encode("utf-8"), IV=iv.encode("utf-8"),mode=AES.MODE_CBC)
    # 加密，加密的内容必须是16的倍数
    bs = aes.encrypt(data.encode('utf-8'))
    return str(b64encode(bs), 'utf-8')

# 将字典变为字符串
import json


resp = requests.post(url=url, data={
    "params": get_params(json.dumps(data)),
    "encSecKey": encSecKey
})

comments = resp.json()['data']['comments']

# print(resp.json())

with open('./5-5网易云评论数据.txt', 'w', encoding='utf-8') as fp:
    for item in comments:
        fp.write(item['user']['nickname']+':'+item['content'] + '\n')
    print("爬取成功")



爬取成功
